In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
pd.__version__

'1.1.5'

In [1]:
!pwd    # 느낌표 다음에 실행되는거는 유닉스 명령어 실행
        # 현재 디렉토리를 알려줌(!pwd)
# !ls drive/MyDrive
        # 디렉토리 내의 모든 파일을 나열함(!ls)

/content


In [ ]:
data_file1 = "/content/drive/MyDrive/data/아파트(매매)__실거래가_(20190701_20200531).csv"
# csv파일은 콤마로 분리된 데이터파일을 뜻함

# Indexing

> 들여쓴 블록



## data 불러오기

- ```python
    >>> pd.read_csv(file_name, skiprows=..., encoding=...)
  ```

In [ ]:
df = pd.read_csv(data_file1, skiprows=15, encoding='euc-kr')
# skiprows : csv파일에 실직적인 부분만 파악하기 위해 필요없는 부분은 건너뛸 수 있다.
df.head(4)      # 위에서 n번째를 보여줌
df.tail(3)      # 끝에서 n번째를 보여줌

In [ ]:
x=[[1,2,3],[4,5,6]]
print(x[1][2])

In [ ]:
df.shape    # 몇 by 몇 matrix인지 알려줌

In [ ]:
df.dtypes   #각 column마다 어떤 자료형인지 알려줌

In [ ]:
df.index

In [ ]:
df.columns

In [ ]:
df.to_numpy()    # numpy배열로 만들어줌

### ex) 데이터의 하단 10개를 출력해 보자.

- hint: `df.tail`

In [ ]:
df.tail(10)

---

## Index 설정

In [ ]:
apt_df = df.set_index("단지명")     # 인덱스를 단지명으로 바꿔줌(0~32299에서 건영맨션,태평라이프 등등으로 바뀜)
apt_df.tail()

In [ ]:
# 되돌릴 때
# apt_df.reset_index()

## 1. Label based indexing

- `.loc` 를 사용한다.
    ```python
    >>> df.loc[row_labels, column_labels]
    ```

In [ ]:
apt_df.loc["대구테크노폴리스남해오네뜨1차"]

In [ ]:
apt_df.loc[:, ["계약년월", "시군구"]]

### ex) 
마음에 드는 두 아파트에 대한 거래 금액과 전용면적을 출력해보자

In [ ]:
apt_df.loc[["개나리맨션","태평라이프"],["전용면적(㎡)", "거래금액(만원)"]]

## 2. Position based indexing

- `.iloc` 를 사용한다.
    ```python
    >>> df.iloc[row_positions, column_positions]
    ```

In [ ]:
apt_df.iloc[0]      # 첫번째 행이 출력됨

In [ ]:
apt_df.iloc[[0]]    # 위의 결과와는 다르게 matrix(표)로 출력됨

In [ ]:
apt_df.iloc[0:1]    # [[0]]과 [0:1]은 같은 인덱스를 뜻한다.

In [ ]:
apt_df.iloc[:, 1:5]     

In [ ]:
apt_df.iloc[1, 1]       # 첫번째 행과 첫번째 열을 뜻하는데 젤 위의 변수명은 빼고 생각한다!!

### ex)
`iloc` 을 사용해서 `apt_df` 데이터 프레임의 `'전용면적(㎡)'`, `'거래금액(만원)'` 두 컬럼을 출력한다. 단 이 때 출력한 행은 마지막 10개의 행으로 제한한다.

In [ ]:
# 직접 position count
print(apt_df.iloc[-10:, [4,7]])

# column position 직접 count하지 않고 코드로 작성 & list_comprehension 사용
col_idx = [apt_df.columns.get_loc(col_name) for col_name in ["전용면적(㎡)", "거래금액(만원)"]]
print(col_idx)
print(apt_df.iloc[-10: ,col_idx])

## 3. dict_like indexing (`__getitem__`)

- `[]`을 사용한다.
    ```python
    >>> df[['col1', 'col2']]
    ```

In [ ]:
apt_df[['거래금액(만원)']]

### ex)

`시군구`, `계약년월` 열을 선택해보자

In [ ]:
apt_df[["시군구", "계약년월"]]

## 4. Boolean Indexing

In [ ]:
df['해제사유발생일'].isna()

In [ ]:
df['해제사유발생일'].isnull()       # 파이썬에서는 isnull() = isna()

In [ ]:
mask = df['해제사유발생일'].isna() # df['해제사유발생일'].notna()
                                   # notna() : 결측값은 False 그 외에는 True    (결측값이란 비어있는 값)
                                   # isna() : 결측값은 True 그 외에는 False
df.loc[mask]        # mask : 해제사유발생일이 NaN인 것들을 모아둔 것

### logical operator

- `~` for **not**
- `&` for **and**
- `|` for **or**

'해제사유발생일' 이 값이 `NaN` 이 아닌것 선택하기

In [ ]:
df.loc[~mask]       # ~mask : not mask = not NaN

### 두가지 조건 만족하는 row 골라내기

- '해제사유발생일' 값이 있을것
- 건축년도가 2020년이다.

In [ ]:
year_condition = df['건축년도'] == 2020

df.loc[year_condition & ~mask]

### ex)
단지명이 `'대구역센트럴자이'` 이거나 또는 `해제사유발생일`이 존재하는 데이터를 골라보자

In [ ]:
apt_condition = df["단지명"] == "대구역센트럴자이"
df.loc[apt_condition | ~mask]


```python
>>> df.isin()
```

In [ ]:
apt_mask = df['단지명'].isin(['개나리맨션', '세종아파트'])
df.loc[apt_mask]['거래금액(만원)']

## 5. 필요없는 부분 제거(drop) 하기

- na 가 하나라도 존재하는 행 제거

In [ ]:
df.dropna(axis="index", how='any')      # dropna(how='any')이면 어떤 행에 Nan이 하나라도 있으면 drop한다는 뜻
                                        # dropna(how='all')이면 행 안에 모든 것이 Nan이면 drop한다는 뜻

### ex)

na 가 하나라도 존재하는 열 제거

In [ ]:
df.dropna(axis='columns', how='any')

- 명시적으로 drop 하기

In [ ]:
df.drop(['해제사유발생일'], axis="columns")     # column이라는 곳에서 해제사유발생일을 찾아서 없앤다는 뜻
                                                # df.drop 어디 변수에 저장 안할 경우에는 복사본이 만들어진다고 생각하기! 원형 df는 바뀌지 않는다.

In [ ]:
df

### ex)

`해제사유발생일`, `번지`, `본번`, `부번` 은 열은 이제 다시는 사용할 일이 없다. 이 열들을 drop 해보자.

In [ ]:
df = df.drop(["해제사유발생일", "번지", "본번", "부번"], axis="columns")
# df.drop(["해제사유발생일", "번지", "본번", "부번"], axis="columns",inplace=True) 위의 실행결과와 같다.
# inplace=False이면 return a copy, inplace=True이면 do operation inplace and return None

In [ ]:
df

--- 

## 6. data column 추가하기

### column 명 바꾸기

In [ ]:
rename_map = {
    "계약년월":'yearmonth',
    "계약일": 'date',
    "시군구":"address",
    "단지명":"apt",
    "전용면적(㎡)":"area",
    "거래금액(만원)":"transaction",
    "층":"floor",
    "건축년도":"built",
    "도로명":"road"
}
df.rename(
    rename_map, axis="columns", inplace=True
)
# rename(inplace=True)이면 진짜 바꿀거니까 복사본을 만드는게 아니라 실제로 바꾸라는 의미

In [ ]:
df

datetime format

- `%Y` : 2020, 2021
- `%m` : 01, 02, ... 12
- `%d` : 01, 02, ... 31

In [ ]:
import datetime     # datetime 모듈을 불러옴

time_data = datetime.datetime.strptime("2020-06-01","%Y-%m-%d")     # %Y : 연도가 네자리 수 인 연도 표시
# datetime 모듈 안의 datetime 클래스는 날짜와 시간을 동시에 표시할 수 있다.
# datetime(연, 월, 일, 시, 분, 초, (마이크로 초))
# strptime() 메서드는 다양한 포멧의 문자열을 datetime 객체로 변환해줍니다.
time_data

In [ ]:
time_data.date()

In [ ]:
time_data.year

In [ ]:
time_data.day

In [ ]:
df.head()       # default 값이 5

### 계약날짜 column 만들기

`yearmonth`와 `date` 열을 합쳐서 하나로 만들고 pandas 의 시계열 자료형으로 바꾸자

---


시계열 자료 : 인덱스가 날짜 혹은 시간인 데이터

In [ ]:
# pandas의 to_datetime은 날짜 또는 시간을 나타내는 문자열을 자동으로 datetime 자료형으로 바꾸고 DatetimeIndex 자료형 인덱스를 생성한다.
# astype('str') : 모든 열의 데이터타입을 ()안에 있는 자료형으로 변경
df["cdate"] = pd.to_datetime(df['yearmonth'].astype('str') + df['date'].astype('str'), format="%Y%m%d")

# 위의 format="%Y%m%d"은 읽어오는 format이고 적히는 형식은 "Y-m-d"으로 적힌다.

In [ ]:
df

### ex)
- `계약년월`, `계약일` column을 drop 해보자

In [ ]:
df = df.drop(["yearmonth", "date"], axis="columns")

In [ ]:
df

### 계약날짜를 년, 월, 날짜로 나누어 새로운 column으로 추가

In [ ]:
#dataframe.series.dt(distribution).year : 한 column을 그 column내의 요소들로 나누어 새로운 column들을 만들 수 있다.
df['year'] = df.cdate.dt.year
df['month'] = df.cdate.dt.month
df['day'] = df.cdate.dt.day

In [ ]:
df.head(5)

### 주소를 각각 시, 구, 동 으로 나고 이를 column으로 저장

In [ ]:
df[['city', 'gu', 'dong']] = df.address.str.split(' ', n=2, expand=True)

In [ ]:
df.head(5)

In [ ]:
df = df.drop(['address'], axis='columns')

In [ ]:
df

---

### 매매금액(`transaction`)을 숫자형 자료로 바꾸자

- `DataFrame.apply`: 전체 row, column 에 적용
- `Series.map`, `Series.apply`, `DataFrame.applymap`: 각각의 요소마다 적용(elementwise)

In [ ]:
df.head(2)

In [ ]:
# lambda x: x(return값) : small task를 표현할때 사용
# lambda x: x+3 vs def(add3(x)): return x+3

df['transaction'].map(lambda string: string.replace(',', ''))   # 3자리 마다 (,) 적힌걸 없애는 과정

In [ ]:
# dataFrame에서 복수 개의 column이 필요하다면 apply함수를 사용

df['transaction'].apply(lambda string: string.replace(',', ''))

In [ ]:
# applymap() 함수는 dataframe클래스의 함수이긴 하지만 
# 위의 apply함수처럼 각 row(axis=1)나 각 column(axis=0)별로 작동하는 함수가 아니라, 각 요소(element)별로 작동한다.
# 마치 선형대수에서 벡터에 스칼라를 연산하면, 벡터의 요소 하나하나에 해당 연산을 해주는 것처럼

df[['transaction']].applymap(lambda string: string.replace(',', ''))

In [ ]:
df['transaction'] = df['transaction'].str.replace(',', '').astype(int)      # string이였던 자료형을 int형으로 바꾸었다.

In [ ]:
df

### `cdate`(계약날짜) 를 새 인덱스로 설정한후 파일로 저장하자

In [ ]:
df.set_index('cdate').to_csv("apt_trade_part1.csv")     #df.set_index로 인덱스 지정후, to_csv로 csv파일로 저장

In [ ]:
pd.read_csv("apt_trade_part1.csv")

In [ ]:
df.cdate.dtype.type